In [3]:
import pandas as pd
# import vaex
import numpy as np
import glob
import dask.dataframe as dd
import json
from sklearn.model_selection import train_test_split
import math
import csv
from sklearn.metrics import accuracy_score, recall_score, f1_score, precision_score, classification_report, confusion_matrix
import time
import _warnings
import tensorflow as tf
from tqdm import tqdm
import swifter
import argparse
import os
import sys
import helper_functions
from importlib import reload

In [2]:
%load_ext autoreload
%autoreload 2

sys.path.append('../')
sys.path.insert(0, os.path.dirname(os.getcwd()) + "/../")

print(sys.path)

['/home/tiendat/transformer-entropy-ids/../', '/home/tiendat/transformer-entropy-ids/notebooks', '/home/tiendat/miniconda3/envs/torchtf/lib/python39.zip', '/home/tiendat/miniconda3/envs/torchtf/lib/python3.9', '/home/tiendat/miniconda3/envs/torchtf/lib/python3.9/lib-dynload', '', '/home/tiendat/miniconda3/envs/torchtf/lib/python3.9/site-packages', '../']


In [4]:
#os.chdir(os.path.dirname(os.getcwd()) + "/road/attacks")
os.chdir(os.path.dirname(os.getcwd()) + "/road/ambient")
print(os.getcwd())

/home/tiendat/transformer-entropy-ids/road/ambient


In [7]:
def get_all_data(attack_dict):
    df_aggregation = []
    
    for attack_name, metadata in attack_dict.items():    
        if "masquerade" not in attack_name and "accelerator" not in attack_name and "metadata" not in attack_name:
            file_name = '/home/tiendat/transformer-entropy-ids/road/attacks/{}.log'.format(attack_name)
            df_attack = helper_functions.make_can_df(file_name)
            df_attack = helper_functions.add_time_diff_per_aid_col(df_attack)
            # print(df_attack.shape)
            # print(df.dtypes)
            df_aggregation.append(df_attack)
            print(f"Finish preprocess {file_name}")
    return df_aggregation

def get_time_interval(attack_dict):
    attack_metadata = []
    
    for attack_name, metadata in attack_dict.items():    
        if "masquerade" not in attack_name and "accelerator" not in attack_name and "metadata" not in attack_name:
            print(f"Finish get time interval of {attack_name}")
            # print(count, file_name)
            
            # if attack_dict[file_name[:-4]]["injection_id"] != "XXX":
            #     print(attack_dict[file_name[:-4]]["injection_id"], int(attack_dict[file_name[:-4]]["injection_id"], 16))
            # else:
            #     print(attack_dict[file_name[:-4]]["injection_id"])
            
            # print(attack_dict[file_name[:-4]]["injection_id"], int(attack_dict[file_name[:-4]]["injection_id"], 16))
            
            # attack_metadata.append(helper_functions.get_injection_interval(df_aggregation[count], int(attack_dict[file_name[:-4]]["injection_id"], 16),
            #                                                              attack_dict[file_name[:-4]]["injection_data_str"]))
            
            # From metadata file
            attack_metadata.append([tuple(attack_dict[attack_name]["injection_interval"])])
    return attack_metadata

def mark_label(df_aggregation, attack_metadata, attack_dict):
    count = 0
    for attack_name, metadata in attack_dict.items():    
        if "masquerade" not in attack_name and "accelerator" not in attack_name and "metadata" not in attack_name:
            print(f"Index {count}: {attack_name} --- {attack_dict[attack_name]['injection_id']}")
            
            # print(attack_dict[file_name[:-4]]["injection_id"]) #, int(attack_dict[file_name[:-4]]["injection_id"], 16))#, attack_dict[file_name[:-4]]["injection_data_str"])
            
            # Add column to each attack dataframe to indicate attack (True) or non-attack (False) for each signal
            
            if attack_dict[attack_name]["injection_id"] != "XXX":
                df_aggregation[count] = helper_functions.add_actual_attack_col(df_aggregation[count], attack_metadata[count], int(attack_dict[attack_name]["injection_id"], 16), attack_dict[attack_name]["injection_data_str"], attack_name)
                print(len(df_aggregation[count][df_aggregation[count]['label'] == True]['label']))
                print(len(df_aggregation[count][df_aggregation[count]['label'] == False]['label']))
            else:
                df_aggregation[count] = helper_functions.add_actual_attack_col(df_aggregation[count], attack_metadata[count], "XXX", attack_dict[attack_name]["injection_data_str"], attack_name)
                print(len(df_aggregation[count][df_aggregation[count]['label'] == True]['label']))
                print(len(df_aggregation[count][df_aggregation[count]['label'] == False]['label']))
            count += 1
    return df_aggregation

def filter_attack(arr, keyword):
    sub_array = []
    for item in arr:
        if keyword in item:
            sub_array.append(item)
    return sub_array

In [8]:
with open("/home/tiendat/transformer-entropy-ids/road/attacks/capture_metadata.json", "r") as read_file:
    attack_dict = json.load(read_file)

# display(attack_dict)

df_aggregation = get_all_data(attack_dict)
attack_metadata = get_time_interval(attack_dict)
df_aggregation = mark_label(df_aggregation, attack_metadata, attack_dict)

display(df_aggregation[0])
# for attack in attacks: 
#     print(f"We got {attack} at index {count}")
#     sub_attacks = filter_attack(attack_file, attack) # got list of attack ... fuzzing,...
#     print(sub_attacks)
    
#     print("Now we put to csv")
#     df_aggregation = []
#     sum_lenght = 0
#     for data in sub_attacks:
#         df_attack = helper_functions.make_can_df(data)
#         df_attack = helper_functions.add_time_diff_per_aid_col(df_attack)
#         print(df_attack.shape)
#         sum_lenght += df_attack.shape[0]
#         # print(df.dtypes)
#         df_aggregation.append(df_attack)
#         count += 1 
#     # df_res = pd.concat(df_aggregation)
#     # display(df_res)
#     print(f"Total of this boy {attack} is {sum_lenght}\n")
#     for attack in attacks: 
#         if attack in file_name and "masquerade" not in file_name and "accelerator" not in file_name and "metadata" not in file_name:
#             print(file_name)
            
#             df_attack = helper_functions.make_can_df(file_name)
#             # print(df_attack.shape)
#             # print(df.dtypes)
#             df_aggregation.append(df_attack)
#     df_res = pd.concat(df_aggregation)
#     df_aggregation = []
#     display(df_res)
# print(len(df_aggregation))

Finish preprocess /home/tiendat/transformer-entropy-ids/road/attacks/correlated_signal_attack_1.log
Finish preprocess /home/tiendat/transformer-entropy-ids/road/attacks/correlated_signal_attack_2.log
Finish preprocess /home/tiendat/transformer-entropy-ids/road/attacks/correlated_signal_attack_3.log
Finish preprocess /home/tiendat/transformer-entropy-ids/road/attacks/fuzzing_attack_1.log
Finish preprocess /home/tiendat/transformer-entropy-ids/road/attacks/fuzzing_attack_2.log
Finish preprocess /home/tiendat/transformer-entropy-ids/road/attacks/fuzzing_attack_3.log
Finish preprocess /home/tiendat/transformer-entropy-ids/road/attacks/max_engine_coolant_temp_attack.log
Finish preprocess /home/tiendat/transformer-entropy-ids/road/attacks/max_speedometer_attack_1.log
Finish preprocess /home/tiendat/transformer-entropy-ids/road/attacks/max_speedometer_attack_2.log
Finish preprocess /home/tiendat/transformer-entropy-ids/road/attacks/max_speedometer_attack_3.log
Finish preprocess /home/tiendat/

,time,aid,data,ori_time,time_diffs,label
3873,1.618163,6,0800006400000000,1.030000e+09,0.999845,False
6266,2.618064,6,0800006400000000,1.030000e+09,0.999901,False
8655,3.617806,6,0800006400000000,1.030000e+09,0.999742,False
11049,4.617810,6,0800006400000000,1.030000e+09,1.000004,False
13441,5.618164,6,0800006400000000,1.030000e+09,1.000354,False
...,...,...,...,...,...,...
80160,32.636582,1788,00000738D2B85800,1.030000e+09,0.099969,False
80402,32.736450,1788,0000073732B86000,1.030000e+09,0.099868,False
80641,32.836612,1788,0000073892B86800,1.030000e+09,0.100162,False
80874,32.936532,1788,00000737B2B87000,1.030000e+09,0.099920,False


In [63]:
count = 0
outdir = '/home/tiendat/transformer-entropy-ids/road/co_process/'
for attack_name, metadata in attack_dict.items():
    if "masquerade" not in attack_name and "accelerator" not in attack_name and "metadata" not in attack_name:
        print(f"Save into {attack_name}_dataset.csv")
        foutput = '{}/{}_dataset.csv'.format(outdir, attack_name)
        df_aggregation[count].to_csv(foutput, index=False)
        count += 1

Save into correlated_signal_attack_1_dataset.csv
Save into correlated_signal_attack_2_dataset.csv
Save into correlated_signal_attack_3_dataset.csv
Save into fuzzing_attack_1_dataset.csv
Save into fuzzing_attack_2_dataset.csv
Save into fuzzing_attack_3_dataset.csv
Save into max_engine_coolant_temp_attack_dataset.csv
Save into max_speedometer_attack_1_dataset.csv
Save into max_speedometer_attack_2_dataset.csv
Save into max_speedometer_attack_3_dataset.csv
Save into reverse_light_off_attack_1_dataset.csv
Save into reverse_light_off_attack_2_dataset.csv
Save into reverse_light_off_attack_3_dataset.csv
Save into reverse_light_on_attack_1_dataset.csv
Save into reverse_light_on_attack_2_dataset.csv
Save into reverse_light_on_attack_3_dataset.csv


In [58]:
# List the files in the 'data' folder
[file for file in os.listdir("/home/tiendat/transformer-entropy-ids/road/ambient") if "dyno" in file]

['ambient_dyno_exercise_all_bits.log',
 'ambient_dyno_drive_extended_short.log',
 'ambient_dyno_drive_basic_short.log',
 'ambient_dyno_drive_benign_anomaly.log',
 'ambient_dyno_idle_radio_infotainment.log',
 'ambient_dyno_reverse.log',
 'ambient_dyno_drive_radio_infotainment.log',
 'ambient_dyno_drive_extended_long.log',
 'ambient_dyno_drive_basic_long.log',
 'ambient_dyno_drive_winter.log']

In [ ]:
import helper_functions

df_aggregation = []
df_large = []

for file_name in os.listdir("/home/tiendat/transformer-entropy-ids/road/ambient"):
    
    if "dyno" in file_name:
        
        print(file_name)
        df = helper_functions.make_can_df(file_name)
        df = helper_functions.add_time_diff_per_aid_col(df)
        print(df.shape)
        # display(df[df['time_diffs'] > 10])
        # print(df.dtypes)
        df_large.append(df[df['time_diffs'] > 10])
        df_aggregation.append(df)

# Concatenate all training datasets on the dyno
#df_training = pd.concat(df_aggregation)
#df_training['label'] = [False] * 13300759
# df_training = df_training[["time", "aid", "time_diffs"]]
#display(df_training)

In [91]:
df_anomaly = pd.concat(df_aggregation)
df_anomaly = df_anomaly[df_anomaly['label'] == True]
# display(df_anomaly[df_anomaly['time_diffs'] > 10])

display(df_anomaly[df_anomaly['time_diffs'] > 1])

,time,aid,data,ori_time,time_diffs,label


In [23]:
df_training.to_csv("/home/tiendat/transformer-entropy-ids/road/co_process/aggregated_training_data.csv", index=False)

In [72]:
import pandas as pd

# Load the CSV file into a DataFrame
file_path = '/home/tiendat/transformer-entropy-ids/road/co_process/aggregated_training_data.csv'
df = pd.read_csv(file_path)

# Masquerade

In [9]:
def get_all_data(attack_dict):
    df_aggregation = []
    
    for attack_name, metadata in attack_dict.items():    
        if "accelerator" not in attack_name and "metadata" not in attack_name:
            print(f"{attack_name}")
            file_name = '/home/tiendat/transformer-entropy-ids/road/attacks/{}.log'.format(attack_name)
            df_attack = helper_functions.make_can_df(file_name)
            df_attack = helper_functions.add_time_diff_per_aid_col(df_attack)
            # print(df_attack.shape)
            # print(df.dtypes)
            df_aggregation.append(df_attack)
            print(f"Finish preprocess {file_name}")
    return df_aggregation

def get_time_interval(attack_dict):
    attack_metadata = []
    
    for attack_name, metadata in attack_dict.items():    
        if "accelerator" not in attack_name and "metadata" not in attack_name:
            print(f"Finish get time interval of {attack_name}")
            # print(count, file_name)
            
            # if attack_dict[file_name[:-4]]["injection_id"] != "XXX":
            #     print(attack_dict[file_name[:-4]]["injection_id"], int(attack_dict[file_name[:-4]]["injection_id"], 16))
            # else:
            #     print(attack_dict[file_name[:-4]]["injection_id"])
            
            # print(attack_dict[file_name[:-4]]["injection_id"], int(attack_dict[file_name[:-4]]["injection_id"], 16))
            
            # attack_metadata.append(helper_functions.get_injection_interval(df_aggregation[count], int(attack_dict[file_name[:-4]]["injection_id"], 16),
            #                                                              attack_dict[file_name[:-4]]["injection_data_str"]))
            
            # From metadata file
            attack_metadata.append([tuple(attack_dict[attack_name]["injection_interval"])])
    return attack_metadata

def mark_label(df_aggregation, attack_metadata, attack_dict):
    count = 0
    for attack_name, metadata in attack_dict.items():    
        if "accelerator" not in attack_name and "metadata" not in attack_name:
            print(f"Index {count}: {attack_name} --- {attack_dict[attack_name]['injection_id']}")
            
            # print(attack_dict[file_name[:-4]]["injection_id"]) #, int(attack_dict[file_name[:-4]]["injection_id"], 16))#, attack_dict[file_name[:-4]]["injection_data_str"])
            
            # Add column to each attack dataframe to indicate attack (True) or non-attack (False) for each signal
            
            if attack_dict[attack_name]["injection_id"] != "XXX":
                df_aggregation[count] = helper_functions.add_actual_attack_col(df_aggregation[count], attack_metadata[count], int(attack_dict[attack_name]["injection_id"], 16), attack_dict[attack_name]["injection_data_str"], attack_name)
                print(len(df_aggregation[count][df_aggregation[count]['label'] == True]['label']))
                print(len(df_aggregation[count][df_aggregation[count]['label'] == False]['label']))
            else:
                df_aggregation[count] = helper_functions.add_actual_attack_col(df_aggregation[count], attack_metadata[count], "XXX", attack_dict[attack_name]["injection_data_str"], attack_name)
                print(len(df_aggregation[count][df_aggregation[count]['label'] == True]['label']))
                print(len(df_aggregation[count][df_aggregation[count]['label'] == False]['label']))
            count += 1
    return df_aggregation

def filter_attack(arr, keyword):
    sub_array = []
    for item in arr:
        if keyword in item:
            sub_array.append(item)
    return sub_array

In [10]:
with open("/home/tiendat/transformer-entropy-ids/road/attacks/capture_metadata.json", "r") as read_file:
    attack_dict = json.load(read_file)

# display(attack_dict)

df_aggregation = get_all_data(attack_dict)
attack_metadata = get_time_interval(attack_dict)
df_aggregation = mark_label(df_aggregation, attack_metadata, attack_dict)

display(df_aggregation[0])

correlated_signal_attack_1
Finish preprocess /home/tiendat/transformer-entropy-ids/road/attacks/correlated_signal_attack_1.log
correlated_signal_attack_1_masquerade
Finish preprocess /home/tiendat/transformer-entropy-ids/road/attacks/correlated_signal_attack_1_masquerade.log
correlated_signal_attack_2
Finish preprocess /home/tiendat/transformer-entropy-ids/road/attacks/correlated_signal_attack_2.log
correlated_signal_attack_2_masquerade
Finish preprocess /home/tiendat/transformer-entropy-ids/road/attacks/correlated_signal_attack_2_masquerade.log
correlated_signal_attack_3
Finish preprocess /home/tiendat/transformer-entropy-ids/road/attacks/correlated_signal_attack_3.log
correlated_signal_attack_3_masquerade
Finish preprocess /home/tiendat/transformer-entropy-ids/road/attacks/correlated_signal_attack_3_masquerade.log
fuzzing_attack_1
Finish preprocess /home/tiendat/transformer-entropy-ids/road/attacks/fuzzing_attack_1.log
fuzzing_attack_2
Finish preprocess /home/tiendat/transformer-entr

,time,aid,data,ori_time,time_diffs,label
3873,1.618163,6,0800006400000000,1.030000e+09,0.999845,False
6266,2.618064,6,0800006400000000,1.030000e+09,0.999901,False
8655,3.617806,6,0800006400000000,1.030000e+09,0.999742,False
11049,4.617810,6,0800006400000000,1.030000e+09,1.000004,False
13441,5.618164,6,0800006400000000,1.030000e+09,1.000354,False
...,...,...,...,...,...,...
80160,32.636582,1788,00000738D2B85800,1.030000e+09,0.099969,False
80402,32.736450,1788,0000073732B86000,1.030000e+09,0.099868,False
80641,32.836612,1788,0000073892B86800,1.030000e+09,0.100162,False
80874,32.936532,1788,00000737B2B87000,1.030000e+09,0.099920,False


In [104]:
df_allattack = pd.concat(df_aggregation)
df_allattack = df_allattack[df_allattack['label'] == False]

,time,aid,data,ori_time,time_diffs,label
3873,1.618163,6,0800006400000000,1.030000e+09,0.999845,False
6266,2.618064,6,0800006400000000,1.030000e+09,0.999901,False
8655,3.617806,6,0800006400000000,1.030000e+09,0.999742,False
11049,4.617810,6,0800006400000000,1.030000e+09,1.000004,False
13441,5.618164,6,0800006400000000,1.030000e+09,1.000354,False
...,...,...,...,...,...,...
152598,63.791930,1788,0000075610B75820,1.280000e+09,0.100035,False
152832,63.891872,1788,0000075670B75820,1.280000e+09,0.099942,False
153067,63.991885,1788,0000075630B75820,1.280000e+09,0.100013,False
153310,64.091944,1788,0000075670B75820,1.280000e+09,0.100059,False


In [115]:
# display(df_allattack)
# display(df_training)
df_test = []
df_test.append(df_allattack)
df_test.append(df_training)
df_all = pd.concat(df_test)

display(df_all)

,time,aid,data,ori_time,time_diffs,label
3873,1.618163,6,0800006400000000,1.030000e+09,0.999845,False
6266,2.618064,6,0800006400000000,1.030000e+09,0.999901,False
8655,3.617806,6,0800006400000000,1.030000e+09,0.999742,False
11049,4.617810,6,0800006400000000,1.030000e+09,1.000004,False
13441,5.618164,6,0800006400000000,1.030000e+09,1.000354,False
...,...,...,...,...,...,...
113058,47.280774,1788,0000077B90C2C000,1.050000e+09,0.099978,NaN
113297,47.381792,1788,0000077B10C2B800,1.050000e+09,0.101018,NaN
113533,47.481730,1788,0000077990C2B800,1.050000e+09,0.099938,NaN
113783,47.581725,1788,0000077AD0C2B800,1.050000e+09,0.099995,NaN


In [113]:
display(df_all[df_all['time_diffs'] > 10].sort_values('aid'))

dasd = df_all[df_all['time_diffs'] > 10].sort_values('aid')



,time,aid,data,ori_time,time_diffs,label
136947,187.059267,6,0000000000000000,1.060000e+09,47.115065,NaN
202776,265.830516,6,0800006400000000,1.050000e+09,15.903907,NaN
190684,240.764793,6,0000000000000000,1.050000e+09,18.179140,NaN
30729,79.904090,6,0000000000000000,1.050000e+09,27.411714,NaN
21280,43.290646,6,0800006400000000,1.050000e+09,12.470706,NaN
...,...,...,...,...,...,...
299571,348.272598,1788,0000175300800040,1.050000e+09,47.230318,NaN
478865,523.385481,1788,0000175300800040,1.050001e+09,139.883811,NaN
137880,188.190132,1788,0000175300800040,1.060000e+09,178.678979,NaN
16785,20.113258,1788,0000175300800040,1.040000e+09,19.288755,NaN


In [28]:
count = 0
outdir = '/home/tiendat/transformer-entropy-ids/road/masquerade_fabrication/'
for attack_name, metadata in attack_dict.items():
    if "accelerator" not in attack_name and "metadata" not in attack_name:
        print(f"Save into {attack_name}_dataset.csv")
        foutput = '{}/{}_dataset.csv'.format(outdir, attack_name)
        df_aggregation[count].to_csv(foutput, index=False)
        count += 1

Save into correlated_signal_attack_1_dataset.csv
Save into correlated_signal_attack_1_masquerade_dataset.csv
Save into correlated_signal_attack_2_dataset.csv
Save into correlated_signal_attack_2_masquerade_dataset.csv
Save into correlated_signal_attack_3_dataset.csv
Save into correlated_signal_attack_3_masquerade_dataset.csv
Save into fuzzing_attack_1_dataset.csv
Save into fuzzing_attack_2_dataset.csv
Save into fuzzing_attack_3_dataset.csv
Save into max_engine_coolant_temp_attack_dataset.csv
Save into max_engine_coolant_temp_attack_masquerade_dataset.csv
Save into max_speedometer_attack_1_dataset.csv
Save into max_speedometer_attack_1_masquerade_dataset.csv
Save into max_speedometer_attack_2_dataset.csv
Save into max_speedometer_attack_2_masquerade_dataset.csv
Save into max_speedometer_attack_3_dataset.csv
Save into max_speedometer_attack_3_masquerade_dataset.csv
Save into reverse_light_off_attack_1_dataset.csv
Save into reverse_light_off_attack_1_masquerade_dataset.csv
Save into rever

In [30]:
log_filepath = '/home/tiendat/transformer-entropy-ids/road/ambient/ambient_dyno_drive_basic_short.log'
can_df = pd.read_fwf(
        log_filepath, delimiter = ' '+ '#' + '('+')',
        skiprows = 0,skipfooter=0,
        usecols = [0,2,3],
        dtype = {0:'float64', 1:str, 2: str},
        names = ['time','aid', 'data'] )

can_df.aid = can_df.aid.apply(lambda x: int(x,16))
can_df.data = can_df.data.apply(lambda x: x.zfill(16))
can_df.time = can_df.time - can_df.time.min()
display(can_df)

,time,aid,data
0,0.000000,354,00080003EA11F4CE
1,0.000002,167,0010FA27012B90A0
2,0.000003,4095,0000000000000000
3,0.000004,1176,84002000F3800009
4,0.001017,1124,0002400000901A54
...,...,...,...
1063619,444.749613,1255,00000013038C0000
1063620,444.750607,339,00000000000C0002
1063621,444.750608,4095,0000000000000000
1063622,444.750609,661,0000000000000040


In [33]:
can_df.sort_values(['aid','time'], inplace=True)
display(can_df)
display(can_df[can_df['aid'] == 354])

can_df['time_diffs_temp'] = can_df['time'].diff()
display(can_df)
display(can_df[can_df['aid'] == 354])


,time,aid,data
1894,0.792426,6,0800006400000000
4287,1.792703,6,0800006400000000
6681,2.792972,6,0800006400000000
9073,3.793214,6,0800006400000000
11468,4.794420,6,0800006400000000
...,...,...,...
1063578,444.731914,4095,0000000000000000
1063600,444.742540,4095,0000000000000000
1063603,444.743538,4095,0000000000000000
1063605,444.744538,4095,0000000000000000


,time,aid,data
0,0.000000,354,00080003EA11F4CE
46,0.018992,354,00080003EA11F4CE
98,0.040042,354,00080003EA11F4CE
157,0.058830,354,00080003EA11F4CE
202,0.080034,354,00080003EA11F4CE
...,...,...,...
1063407,444.656698,354,00080003EA11F4CE
1063450,444.676448,354,00080003EA11F4CE
1063493,444.696458,354,00080003EA11F4CE
1063540,444.716458,354,00080003EA11F4CE


,time,aid,data,time_diffs_temp
1894,0.792426,6,0800006400000000,NaN
4287,1.792703,6,0800006400000000,1.000277
6681,2.792972,6,0800006400000000,1.000269
9073,3.793214,6,0800006400000000,1.000242
11468,4.794420,6,0800006400000000,1.001206
...,...,...,...,...
1063578,444.731914,4095,0000000000000000,0.009367
1063600,444.742540,4095,0000000000000000,0.010626
1063603,444.743538,4095,0000000000000000,0.000998
1063605,444.744538,4095,0000000000000000,0.001000


,time,aid,data,time_diffs_temp
0,0.000000,354,00080003EA11F4CE,-444.750607
46,0.018992,354,00080003EA11F4CE,0.018992
98,0.040042,354,00080003EA11F4CE,0.021050
157,0.058830,354,00080003EA11F4CE,0.018788
202,0.080034,354,00080003EA11F4CE,0.021204
...,...,...,...,...
1063407,444.656698,354,00080003EA11F4CE,0.020250
1063450,444.676448,354,00080003EA11F4CE,0.019750
1063493,444.696458,354,00080003EA11F4CE,0.020010
1063540,444.716458,354,00080003EA11F4CE,0.020000


,time,aid,data,time_diffs_temp
1894,0.792426,6,0800006400000000,NaN
4287,1.792703,6,0800006400000000,1.000277
6681,2.792972,6,0800006400000000,1.000269
9073,3.793214,6,0800006400000000,1.000242
11468,4.794420,6,0800006400000000,1.001206
...,...,...,...,...
1063578,444.731914,4095,0000000000000000,0.009367
1063600,444.742540,4095,0000000000000000,0.010626
1063603,444.743538,4095,0000000000000000,0.000998
1063605,444.744538,4095,0000000000000000,0.001000


In [48]:
mask = can_df.aid == can_df.aid.shift(1) #get bool mask of to filter out first msg of each group

can_df = can_df[mask]

In [57]:
display(can_df[can_df['time_diffs_temp']>1.8])

,time,aid,data,time_diffs_temp
113377,47.380711,1649,E787FBD5DBDDBFFA,28.738170
121230,50.660658,1649,E7BFFBD5DBDDBFFB,1.999943
193133,80.722018,1649,E787FBD5DBDDBFFA,30.061360
257395,107.582335,1649,E787FBD5DBDDBFFA,26.738264
265240,110.862435,1649,E787FBD5DBDDBFFA,3.159129
338374,141.458516,1649,E787FBD5DBDDBFFA,30.013181
401137,167.718620,1649,E787FBD5DBDDBFFA,26.138055
410472,171.618852,1649,E787FBD5DBDDBFFA,3.756873
415255,173.618313,1649,E7BFFBD5DBDDBFFB,1.999461
487048,203.641760,1649,E787FBD5DBDDBFFA,30.023447


In [47]:
data = [[6, 0.000000], [6, 0.000002],[7, 0.000004],[7, 0.000008],[7, 0.000012],[6, 0.000017],[8, 0.000026],[7, 0.000030]]

df_1 = pd.DataFrame(data, columns=['aid', 'time'])
display(df_1)

df_1.sort_values(['aid','time'], inplace=True)
display(df_1)

df_1['time_diffs_temp'] = df_1['time'].diff()
display(df_1)

mask = df_1.aid == df_1.aid.shift(1) #get bool mask of to filter out first msg of each group
display(mask)

df_1 = df_1[mask]
display(df_1)

,aid,time
0,6,0.000000
1,6,0.000002
2,7,0.000004
3,7,0.000008
4,7,0.000012
5,6,0.000017
6,8,0.000026
7,7,0.000030


,aid,time
0,6,0.000000
1,6,0.000002
5,6,0.000017
2,7,0.000004
3,7,0.000008
4,7,0.000012
7,7,0.000030
6,8,0.000026


,aid,time,time_diffs_temp
0,6,0.000000,NaN
1,6,0.000002,0.000002
5,6,0.000017,0.000015
2,7,0.000004,-0.000013
3,7,0.000008,0.000004
4,7,0.000012,0.000004
7,7,0.000030,0.000018
6,8,0.000026,-0.000004


0    False
1     True
5     True
2    False
3     True
4     True
7     True
6    False
Name: aid, dtype: bool

,aid,time,time_diffs_temp
1,6,0.000002,0.000002
5,6,0.000017,0.000015
3,7,0.000008,0.000004
4,7,0.000012,0.000004
7,7,0.000030,0.000018
